## Describing Imaging Systems
<div style="float: left; width: 45%;">
<br>
<ul>
<li> No imaging system is perfect!  <br><br>
<li> Images only provide a reasonable approximation of the underlying object we are trying to capture. <br><br>
<li> For example, medical images typically provide some information about a particular biological property of the tissue at a given position in space, $(x, y)$ <br><br>
</ul>
</div>
<div style="float: right; width: 50%;">
 <img src='images/medical_image_examples.jpg' width = '80%'>
</div>

## Describing Imaging Systems
<div style="float: left; width: 45%;">
<br>
<ul>
    <li> In the following slides we introduce a general formulation for characterising imaging systems.<br><br>
    <li> We deonte the <b>object</b> as $f(\alpha, \beta)$ , and the <b>image</b> as $g(x, y)$<sup>1</sup><br><br>
    <li> We also typically assume that $f(\alpha, \beta) \in \mathbb{R}^{+}$ and $g(x, y) \in \mathbb{R}^{+}$<br><br>
    <li> In its most general sense, the imaging system can be thought of as an operator, $\mathcal{H}$, operating on the opbject to produce the image.<br><br>
</ul>
<hr></hr>
<sup>1. Here we assume we are working with 2D images, but these methods are applicable to 3 or more dimensions as well!</sup>
</div>
<div style="float: right; width: 50%;">
<img src='images/image_object.jpg' width = '100%'>
</div>

## Linear Imaging Systems
<ul>
    <li>Consider now that the same imaging system is used to capture images, $g_{1}(x, y)$ and $g_{2}(x, y)$, of two objects, $f_{1}(\alpha, \beta)$ and $f_{2}(\alpha, \beta)$ respectively.<br><br>
    <li> A <b>linear imaging system</b> is one that satisfies the following property:<br><br>
\begin{align}
\mathcal{H}\left(g_{1} + g_{2}\right) &= \mathcal{H}\left(g_{1}\right) + \mathcal{H}\left(g_{2}\right) \\ 
        &= f_{1} + f_{2}
\end{align}<br>
     <li> For example, in PET imaging, if the concentration of radioactive tracer doubles, then so does the intensity on the final image.  Another example could be doubling of the exposure time when taking a picture (although there will be a limit!)<br><br>
     <li> In practise nearly all imaging systems are considered to be linear.
</ul>

## Point-Spread Function
<div style="float: left; width: 50%;">
<br>
<ul>
    <li> Lets go back to our example of the pin-hole camera with a finite aperture.<br><br>
    <li> What happens if we use a point-source at position $(\alpha, \beta)$ as the object: $f(\alpha', \beta') = \delta(\alpha' - \alpha, \beta' - \beta)$?<br><br>
    <li> We will end up with the <b>point-spread function</b> (PSF) describing the imaging-system for the location of the point source, $h(x, y; \alpha, \beta)$.<br><br>
    <li> The final image will be the sum (integral) of the contribution from all point-sources that make up the object (assuming the imaging system is linear):
    $$
    g(x, y) = \iint\limits_{-\infty}^{\infty} h(x, y; \alpha, \beta)\cdot f(\alpha, \beta)\text{d}\alpha\text{d}\beta
    $$
</ul>
</div>
<div style="float: left; width: 40%;">
    <img src='images/pinhole_projection.jpg' width = '100%'>
</div>

## Shift Invariance
<div style="float: left; width: 50%;">
<br>
<ul>
    <li>If the PSF is <b>shift invariant</b> then $h$ will not change for different positions in the source frame-of-reference.<br><br>
    <li> Instead, the final image will be a <b>convolution</b> of the shift-invariant PSF with the object:<br><br>
        $$
            g(x, y) = \iint\limits_{-\infty}^{\infty} h(x-\alpha, y-\beta)\cdot f(\alpha, \beta)\text{d}\alpha\text{d}\beta
        $$<br>
    <li> In practice nearly all imaging systems are considered to be shift invariant (even if it is know they are not!).
</ul>
</div>